In [1]:
import nltk
import tensorflow as tf
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()

import numpy
from tensorflow.python.framework import ops
import tflearn
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\test\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open("intents.json") as file:
    data = json.load(file)
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    docs_t = []
    docs_p = []
    words = []
    labels = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            ptn = nltk.word_tokenize(pattern)
            words.extend(ptn)
            docs_p.append(ptn)
            docs_t.append(intent["tag"])
        if intent not in labels:
            labels.append(intent["tag"])
    words = [stemmer.stem(w.lower()) for w in words if w != "?" and "!" and "." and "," and ":"]
    words = sorted(list(set(words)))
    labels = sorted(labels)
    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]
    for x, doc in enumerate(docs_p):
        bag = []
        ptn = [stemmer.stem(w) for w in doc]
        for w in words:
            if w in ptn:
                bag.append(1)
            else:
                bag.append(0)
        out_row = out_empty[:]
        out_row[labels.index(docs_t[x])] = 1
        training.append(bag)
        output.append(out_row)
    training = numpy.array(training)
    output = numpy.array(output)
    saved_data = data
    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output,saved_data), f)

ops.reset_default_graph()

In [3]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training,output,n_epoch=1200,batch_size=8,show_metric=True)
model.save(".model.tflearn")


Training Step: 11999  | total loss: 0.00010 | time: 0.056s
| Adam | epoch: 1200 | loss: 0.00010 - acc: 1.0000 -- iter: 72/73
Training Step: 12000  | total loss: 0.00009 | time: 0.062s
| Adam | epoch: 1200 | loss: 0.00009 - acc: 1.0000 -- iter: 73/73
--
INFO:tensorflow:C:\Users\test\OneDrive\Desktop\Chatbot\Chatbot 2\.model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [4]:
def BagOfWords(s, words):
    bag = [0 for _ in range(len(words))]

    swords = nltk.word_tokenize(s)
    swords = [stemmer.stem(word.lower()) for word in swords]

    for wrd in swords:
        for i, w in enumerate(words):
            if w == wrd:
                bag[i] = 1
    return numpy.array(bag)


def Chat():
    print("Start Talking !")
    while True:
        inp = input("You :")
        if inp.lower() == "quit":
            break
        results = model.predict([BagOfWords(inp, words)])[0]
        rsIndex = numpy.argmax(results)
        tag = labels[rsIndex]

        if results[rsIndex] > 0.7:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
        else:
            print("I didn't get it ,ask again")


Chat()

Start Talking !
You :hi
Hi there, how can I help?
You :what's your name
I'm Laeknir!
You :coronavirus
I didn't get it ,ask again
You :whats covid
Coronavirus disease (COVID-19) is an infectious disease caused by the SARS-CoV-2 virus.
Most people who fall sick with COVID-19 will experience mild to moderate symptoms and recover without special treatment. However, some will become seriously ill and require medical attention.
You :quit
